#### **Imports**

In [1]:
import warnings;
warnings.filterwarnings('ignore');

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os, sys
import numpy as np
import random


from  matplotlib.ticker import FuncFormatter # needed to set tickers properly in interactive plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

from ipywidgets import interact
#from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import requests

from settings import PINC_GITHUB_DIR, JSON_CONF_DIR, UPLOAD_DIR, PINC_MV_L
from util.misc import generate_period_list
from util.parse_df import Parse_df
from util.conn_pinc_data import Conn_pinc_data

In [4]:
from stadincijfers import stadincijfers
from gebiedsniveaus.gebiedsniveaus import provincies_dict

#### **Parameters**

In [5]:
all_pinc_periods = True

#### **PinC upload file**

In [6]:
filename = '20210623_Upload crimi en verkeer.xlsx'
#filename = 'NIEUW_huishoudens_Statbel_2020_2071.xlsx'
file = UPLOAD_DIR + '/' + filename
print(file)

./upload_pinc/20210623_Upload crimi en verkeer.xlsx


#### **Import PinC upload file for validation**

In [7]:
# Read excel file, since there was an issue with (a very limited number)
# of values that were not read properly we do a string transformation (using a converter)
# later on (the opposite) int transformation will be applied
column_list = []
df_column = pd.read_excel(file, engine='openpyxl').columns
for j in df_column:
        column_list.append(j)
converter = {col: str for col in column_list}
#print(column_list)

# Read excel, index_col=0 prevents from adding extra index column
# Sheet_name defaults to 0, use other int to choose different or use actual sheet-names (str).
if 'Unnamed' in column_list[0]:
    data_table = pd.read_excel(file, converters=converter, engine='openpyxl', sheet_name=0, index_col=0)
else:
    data_table = pd.read_excel(file, converters=converter, engine='openpyxl', sheet_name=0)

In [8]:
type(data_table)

pandas.core.frame.DataFrame

In [9]:
data_table.head()

,period,geolevel,geoitem,v2802_vh_ver_do_aant,v2802_vh_ver_zgw_aant,v2802_vh_ver_lgw_aant
0,2019,gemeente,11001,3,8,87
1,2019,gemeente,11002,12,176,2369
2,2019,gemeente,11004,0,3,23
3,2019,gemeente,11005,0,4,56
4,2019,gemeente,11007,0,4,42


### **Parse data_table**

In [10]:
parsed_df = Parse_df(data_table)

In [11]:
if all_pinc_periods:
    period_list_pinc_query = parsed_df.determine_years('pinc', _ind_id = 0)
elif not all_pinc_periods:
    period_list_pinc_query = parsed_df.determine_years('table')
else:
    raise Exception("all_pinc_periods requires boolean value")
print(f'Period list for Pinc Query: {period_list_pinc_query}')

List of upload table years: ['2019', '2020']
Period list for Pinc Query: 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020


In [12]:
var_list_pinc_query = parsed_df.determine_indicators()
print(f'Indicator list for PinC Query: {var_list_pinc_query}')

Indicator list for PinC Query: v2802_vh_ver_do_aant,v2802_vh_ver_zgw_aant,v2802_vh_ver_lgw_aant


In [13]:
geolevel = parsed_df.determine_geolevel()
print(f'Geolevel for PinC Query: {geolevel}')

Geolevel for PinC Query: gemeente


### **PinC Query**

In [14]:
# Instantiate object from class
pinc = stadincijfers("provincies")

In [15]:
pinc_table = pinc.selectiontableasDataframe(var_list_pinc_query, geolevel=geolevel, periodlevel='year',period=period_list_pinc_query)

In [16]:
pinc_table.head()

,Geo,Perioden,doden na 30 dagen,zwaargewonden,lichtgewonden
0,Aalst,2009,6.0,28,403
1,Aalst,2010,2.0,33,396
2,Aalst,2011,2.0,31,412
3,Aalst,2012,4.0,29,335
4,Aalst,2013,2.0,22,343


In [17]:
if pinc_table.columns[0] != 'Geo':
    pinc_table.rename(columns={pinc_table.columns[0]: 'Geo'}, inplace = True)

## ***Interactive check framework:***

In [18]:
connection = Conn_pinc_data(pinc_table,data_table)

In [19]:
var_dic = connection.cols_to_dict()
geo_dic = Conn_pinc_data.reversed_level_code_dict(parsed_df.geolevel)

In [20]:
var = random.choice(list(var_dic.keys()))
print(f'Random variable: {var}')
geo = random.choice(list(geo_dic.keys()))
print(f'Random location: {geo}')

Random variable: zwaargewonden
Random location: Gooik


In [21]:
pinc_table.head()

,Geo,Perioden,doden na 30 dagen,zwaargewonden,lichtgewonden
0,Aalst,2009,6.0,28,403
1,Aalst,2010,2.0,33,396
2,Aalst,2011,2.0,31,412
3,Aalst,2012,4.0,29,335
4,Aalst,2013,2.0,22,343


#### **Interactive plots:**

##### **Set additive factor/constant for visually comparing upload and pinc number sequences**

In [22]:
constant = 1

In [23]:
# int transform changed to float transfrom in lambdas
# to do: make constant (to make lines visually different) dynamic
def draw_figure(var= var ,geo= geo):
    tmp_df = pd.DataFrame()
    
    upload_var = var_dic.get(var)
    upload_geo = geo_dic.get(geo)
    
    tmp_df_pinc = pinc_table[pinc_table['Geo']==geo][['Perioden','Geo',var]]
    tmp_df_pinc['Geo'] = tmp_df_pinc['Geo'].apply(lambda x: str(x) + '_PinC')
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) + constant) )  # We add one to ease comparison in plot
    tmp_df = tmp_df.append(tmp_df_pinc)
    #print('PinC Dataframe:')
    #print(tmp_df.head())
    
    tmp_df_upload = data_table[data_table['geoitem']==upload_geo][['period','geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'Geo', 'period': 'Perioden', upload_var : var})
    tmp_df_upload['Geo'] = tmp_df_upload['Geo'].apply(lambda x: str(x) + '_Upload')
    tmp_df_upload[var] = tmp_df_upload[var].apply(lambda x: float(x))
    tmp_df = tmp_df.append(tmp_df_upload)
    #print('Upload Dataframe:')
    #print(tmp_df.head())
    
    tmp_df['Perioden'] = tmp_df['Perioden'].apply(lambda x: int(x))
    #print(tmp_df['Perioden'].unique())
    sns.lineplot(data=tmp_df, x='Perioden',y= var,hue='Geo', style='Geo', markers=True, dashes=False);
    # Format year on xticks to int (instead of float):
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)));
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);


In [24]:
#draw_figure()

In [25]:
list_vars = sorted(list(var_dic.keys()))
list_geos = sorted(list(geo_dic.keys()))  # to do: do an alphabetic sort first

In [26]:
# to do: make better label names
interact(draw_figure, var=list_vars,geo=list_geos);

interactive(children=(Dropdown(description='var', index=2, options=('doden na 30 dagen', 'lichtgewonden', 'zwa…

#### **Outlier analysis**

##### **Overall outlier analysis (make a choice between (A.) automatic generation of comparison years or (B.) setting comparison years yourself):**

In [27]:
# A. Random generation of comparison years (starting from oldest year in upload file):
upload_year = parsed_df.years[0]
print(f'Upload year: {upload_year}')
if str((int(upload_year)-1)) in parsed_df.pinc_q_years :
    pinc_year = int(upload_year)-1
else:
    pinc_year = random.choice(parsed_df.pinc_q_years)
print(f'Pinc year: {pinc_year}')

Upload year: 2019
Pinc year: 2018


In [28]:
# B. Set comparison years:
upload_year = 2020
pinc_year = 2019

In [29]:
data_table.head()

,period,geolevel,geoitem,v2802_vh_ver_do_aant,v2802_vh_ver_zgw_aant,v2802_vh_ver_lgw_aant
0,2019,gemeente,11001,3,8,87
1,2019,gemeente,11002,12,176,2369
2,2019,gemeente,11004,0,3,23
3,2019,gemeente,11005,0,4,56
4,2019,gemeente,11007,0,4,42


In [30]:
tmp_df = pd.DataFrame()
tel = 0
for var in list(var_dic.keys()):


    upload_var = var_dic.get(var)

    tmp_df_pinc = pinc_table[pinc_table['Perioden']==str(pinc_year)][['Geo',var]]
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) ) )
    tmp_df_pinc = tmp_df_pinc[tmp_df_pinc[var]!=-99996]
    tmp_df_pinc = tmp_df_pinc.rename(columns={'Geo':'Geo_pinc',var: '_pinc_' + str(pinc_year)})
    tmp_df_pinc['geo_code_pinc'] = tmp_df_pinc['Geo_pinc'].map(geo_dic)


    tmp_df_upload = data_table[data_table['period']==str(upload_year)][['geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'geo_code_upload', upload_var : '_upload_' + str(upload_year)})
    tmp_df_upload['_upload_' + str(upload_year)] = tmp_df_upload['_upload_' + str(upload_year)].apply(lambda x: float(x))

    merged_df = tmp_df_pinc.merge(tmp_df_upload, left_on='geo_code_pinc', right_on='geo_code_upload')
    merged_df['diff_rel'] = abs(merged_df['_pinc_' + str(pinc_year)]-merged_df['_upload_' + str(upload_year)])/merged_df['_pinc_' + str(pinc_year)]
    merged_df['diff_abs'] = abs(merged_df['_pinc_' + str(pinc_year)]-merged_df['_upload_' + str(upload_year)])
    merged_df['Indicator'] = var + '_P' + str(pinc_year) + '_U' + str(upload_year)
    
    ## Uncomment next line for some validation
    #print(f'Variable: {var}, Shape: {merged_df.shape}')
    tel += merged_df.shape[0]
    tmp_df = tmp_df.append(merged_df)

## Uncomment two next lines for some counters to be used for validation:    
#tel_t = len(list(var_dic.keys())*merged_df.shape[0])     
#print(f'Cumulated no. rows: {tel} vs theoretical no. rows: {tel_t}')


print('Relative difference evaluation')
# Uncomment second line if you want to ignore diff=inf as a result from division by zero
# Uncomment third line if you  also (in addition to previous condition) want to ignore PinC missing values [-99996, -99997, -99998, -99999] in upload file
#tmp_df.sort_values(by=['diff_rel'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff_rel','Indicator']].head(30)
tmp_df[~(tmp_df['diff_rel']==np.inf)].sort_values(by=['diff_rel'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff_rel','Indicator']].head(30)
tmp_df[~((tmp_df['diff_rel']==np.inf) | (tmp_df['_upload_' + str(upload_year)].isin(PINC_MV_L)))].sort_values(by=['diff_rel'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff_rel','Indicator']].head(30)

Relative difference evaluation


,Geo_pinc,_pinc_2019,_upload_2020,diff_rel,Indicator
62,Diksmuide,3.0,16.0,4.333333,zwaargewonden_P2019_U2020
30,Bilzen,3.0,15.0,4.000000,zwaargewonden_P2019_U2020
244,Sint-Agatha-Berchem,1.0,5.0,4.000000,zwaargewonden_P2019_U2020
141,Koekelberg,1.0,5.0,4.000000,zwaargewonden_P2019_U2020
309,Zonhoven,1.0,4.0,3.000000,doden na 30 dagen_P2019_U2020
311,Zottegem,1.0,4.0,3.000000,doden na 30 dagen_P2019_U2020
43,Brecht,1.0,4.0,3.000000,doden na 30 dagen_P2019_U2020
231,Riemst,1.0,3.0,2.000000,zwaargewonden_P2019_U2020
265,Ternat,1.0,3.0,2.000000,zwaargewonden_P2019_U2020
125,Ieper,1.0,3.0,2.000000,doden na 30 dagen_P2019_U2020


In [31]:
print('Absolute difference evaluation')
# Uncomment 2nd line if you want to ignore PinC missing values [-99996, -99997, -99998, -99999] in upload file
tmp_df.sort_values(by=['diff_abs'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff_abs','Indicator']].head(30)
#tmp_df[~(tmp_df['_upload_' + str(upload_year)].isin(PINC_MV_L))].sort_values(by=['diff_abs'], ascending=False)[['Geo_pinc', '_pinc_' + str(pinc_year), '_upload_' + str(upload_year),'diff_abs','Indicator']].head(30)

Absolute difference evaluation


,Geo_pinc,_pinc_2019,_upload_2020,diff_abs,Indicator
8,Antwerpen,2369.0,1831.0,538.0,lichtgewonden_P2019_U2020
81,Gent,1231.0,907.0,324.0,lichtgewonden_P2019_U2020
47,Brussel,1272.0,1022.0,250.0,lichtgewonden_P2019_U2020
46,Brugge,514.0,392.0,122.0,lichtgewonden_P2019_U2020
7,Anderlecht,386.0,272.0,114.0,lichtgewonden_P2019_U2020
209,Oostende,258.0,167.0,91.0,lichtgewonden_P2019_U2020
255,Sint-Niklaas,384.0,299.0,85.0,lichtgewonden_P2019_U2020
148,Kortrijk,307.0,229.0,78.0,lichtgewonden_P2019_U2020
0,Aalst,306.0,229.0,77.0,lichtgewonden_P2019_U2020
164,Leuven,441.0,366.0,75.0,lichtgewonden_P2019_U2020


##### **Interactive/univariate outlier analysis:**

In [32]:
def show_outliers(var = var, upload_year = upload_year, pinc_year = pinc_year):
    upload_var = var_dic.get(var)
    
    tmp_df_pinc = pinc_table[pinc_table['Perioden']==str(pinc_year)][['Geo',var]]
    tmp_df_pinc[var] = tmp_df_pinc[var].apply(lambda x: None if ((not x) or (x=='-') or (x=='x') or (x=='?')) else (float(x) ) ) 
    tmp_df_pinc = tmp_df_pinc.rename(columns={'Geo':'Geo_pinc',var: var + '_pinc_' + str(pinc_year)})
    tmp_df_pinc['geo_code_pinc'] = tmp_df_pinc['Geo_pinc'].map(geo_dic)


    tmp_df_upload = data_table[data_table['period']==str(upload_year)][['geoitem',upload_var]]
    tmp_df_upload = tmp_df_upload.rename(columns={'geoitem':'geo_code_upload', upload_var : var + '_upload_' + str(upload_year)})
    tmp_df_upload[var + '_upload_' + str(upload_year)] = tmp_df_upload[var + '_upload_' + str(upload_year)].apply(lambda x: float(x))

    merged_df = tmp_df_pinc.merge(tmp_df_upload, left_on='geo_code_pinc', right_on='geo_code_upload')
    merged_df['diff'] = abs(merged_df[var + '_pinc_' + str(pinc_year)]-merged_df[var + '_upload_' + str(upload_year)])/merged_df[var + '_pinc_' + str(pinc_year)]
    #return merged_df.sort_values(by=['diff'], ascending=False)[['Geo_pinc', var + '_pinc_' + str(pinc_year), var + '_upload_' + str(upload_year),'diff']].head(20)
    return merged_df[~(merged_df['diff']==np.inf)].sort_values(by=['diff'], ascending=False)[['Geo_pinc', var + '_pinc_' + str(pinc_year), var + '_upload_' + str(upload_year),'diff']].head(20)

In [33]:
#show_outliers()

In [34]:
list_upload_years = parsed_df.years
list_pinc_years = parsed_df.pinc_q_years

In [35]:
# to do: make better label names
interact(show_outliers, var=list_vars,upload_year = list_upload_years, pinc_year = list_pinc_years);

interactive(children=(Dropdown(description='var', index=1, options=('doden na 30 dagen', 'lichtgewonden', 'zwa…